# Projet Recherche Opérationelle - Branch And Bound

## Introduction



Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vivamus consectetur congue nisl, ac laoreet erat mattis vitae. Donec vestibulum vehicula commodo. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Nam nec leo sit amet augue maximus aliquam a id urna. Nam rutrum dictum nulla, in cursus sapien pretium eu. Sed mollis tincidunt nisi at rutrum. Maecenas mauris turpis, gravida tristique pretium a, porta sit amet dolor. Vestibulum vehicula vestibulum tortor ultrices interdum. Praesent vel placerat ipsum, a maximus erat. Nam ac cursus sem. Quisque faucibus tortor venenatis aliquam congue. Vivamus faucibus lacus sed dolor mollis, ac posuere tortor placerat. Donec pellentesque et ligula gravida semper. Quisque cursus ante arcu, vehicula sodales urna imperdiet et. Nunc commodo porttitor interdum. Curabitur ultrices magna ac urna vulputate, vitae pellentesque libero viverra.

Sed eget orci a erat interdum ultrices. Pellentesque est nunc, cursus a rhoncus at, molestie ut justo. Integer ut placerat sapien, sit amet convallis turpis. Curabitur tincidunt tempor iaculis. Cras viverra dignissim purus quis rutrum. Quisque sagittis commodo est id laoreet. Nam varius blandit cursus. Praesent venenatis ultrices nisi ut maximus. Vivamus sed facilisis turpis.

Mauris fermentum lorem vel tincidunt ultrices. Morbi nibh nunc, iaculis ac scelerisque dapibus, malesuada ac ex. Curabitur fermentum interdum felis id aliquet. Cras elementum leo eu accumsan accumsan. Integer vitae aliquet nisl. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Morbi porta libero vitae fermentum convallis. Donec sagittis id sapien et vestibulum. Suspendisse mattis orci a risus egestas, accumsan hendrerit lacus egestas. Proin non purus at lorem tempus fermentum eget ac ipsum. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. In et urna tincidunt eros feugiat rhoncus id sit amet lectus.

Sed sit amet sem tristique, mollis sapien quis, lacinia augue. Sed lorem neque, molestie iaculis imperdiet eu, luctus ac massa. Fusce at augue ut lectus pretium dictum ut ut lacus. Praesent egestas nulla sodales leo porta, eu dapibus dolor pellentesque. Vivamus venenatis accumsan tristique. Vestibulum pellentesque nisi quis massa tincidunt molestie. Sed gravida augue vel lorem blandit, tristique sollicitudin quam aliquam. Ut fermentum rhoncus justo non porttitor. Praesent condimentum lorem in risus porta, eget convallis est dictum. Etiam ac nulla nec ex euismod porttitor. Fusce sed dui porta, faucibus augue at, consectetur eros. Proin suscipit rutrum mi vel ullamcorper. Pellentesque placerat nisi est, eget blandit eros maximus a. Maecenas nulla lacus, pharetra interdum molestie quis, tincidunt ut justo. Cras ultricies mauris eget vehicula accumsan. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus.

Phasellus mattis tellus ante, et mattis turpis pharetra et. Quisque tristique ex et dolor varius ullamcorper. Etiam neque quam, elementum eget neque ac, congue bibendum lorem. Nunc dictum consequat turpis. Pellentesque dolor sapien, gravida quis condimentum egestas, ultricies id augue. Nunc velit lectus, tincidunt eget lacinia sed, finibus quis odio. Interdum et malesuada fames ac ante ipsum primis in faucibus. Cras mattis et massa id aliquam. Suspendisse ornare condimentum varius. 

4


4


4
